# Whatsapp Chat Analysis

In [7]:
import numpy as np
import pandas as pd
import regex
import emoji
from collections import Counter
import matplotlib.pyplot as plt
#from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

## Extraction Functions

In [8]:
def date_time(s):
    pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -'
    result = regex.match(pattern, s)
    if result:
        return True
    return False

def find_author(s):
    s = s.split(":")
    if len(s)==2:
        return True
    else:
        return False

In [9]:
def getDatapoint(line):
    splitline = line.split(' - ')
    dateTime = splitline[0]
    date, time = dateTime.split(", ")
    message = " ".join(splitline[1:])
    if find_author(message):
        splitmessage = message.split(": ")
        author = splitmessage[0]
        message = " ".join(splitmessage[1:])
    else:
        author= None
    return date, time, author, message

## Importing Data

In [10]:
data = []
conversation = 'WhatsApp Chat with Anubharathy.txt'
with open(conversation, encoding="utf-8") as fp:
    fp.readline()
    messageBuffer = []
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        if not line:
            break
        line = line.strip()
        if date_time(line):
            if len(messageBuffer) > 0:
                data.append([date, time, author, ' '.join(messageBuffer)])
            messageBuffer.clear()
            date, time, author, message = getDatapoint(line)
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)

In [11]:
chat_df = pd.DataFrame(data, columns=["Date", 'Time', 'Author', 'Message'])
chat_df['Date'] = pd.to_datetime(chat_df['Date'])

## Exploring Data

In [12]:
chat_df.head()

,Date,Time,Author,Message
0,2021-01-01,10:16 AM,Anubharathy,Tq... Same to u😁
1,2021-01-01,12:15 PM,Veeramanohar,Haa Gm
2,2021-01-01,3:03 PM,Anubharathy,Gm
3,2021-01-01,9:26 PM,Veeramanohar,Oii.. inaiku day epdi pochu ?!
4,2021-01-01,11:05 PM,Anubharathy,Asusual


In [ ]:
char_df.tail